### Transfer learning and fine tuning

#### To classify between Cat and Dogs using pre-trained model 


<html>
<body>
    <h3> Two ways for pretrained model</h3>
<ul>
<li> Feature extraction</li>
<li> Fine Tuning </li>
</ul>
</body>
<html>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

In [ ]:
validation_dataset = image_dataset_from_directory(validation_dir,
                                                 shuffle = True,
                                                 batch_size= BATCH_SIZE,
                                                  image_size = IMG_SIZE)

In [ ]:
class_names = train_dataset.class_names
plt.figure(figsize =(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
#how many batches of data available in validation set
val_batches = tf.data.experimental.cardinality(validation_dataset)#returns the cardinality
test_dataset = validation_dataset.take(val_batches//5) #20%
validation_dataset = validation_dataset.skip(val_batches //5)

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

#### Use data augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    
])

In [ ]:
for image,_ in train_dataset.take(1):
    plt.figure(figsize = (10,10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        augmented_image = data_augmentation(tf.expand_dims(first_image,0))
        plt.imshow(augmented_image[0]/255)
        plt.axis('off')
        
   

### Rescale Pixel values

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale  = tf.keras.layers.experimental.preprocessing.Rescaling(1./127)

In [ ]:
#### Create the base model from the pre-trained convnets

#### base model from pre-trained model MobileNet V2

#### Feature Extraction

In [ ]:
IMG_SHAPE = IMG_SIZE+(3,)
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,
                                              include_top = False,# discards the classification layers at the top
                                              weights = 'imagenet')

In [ ]:
image_batch,label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

#### Feature Extraction

### freeze the convolution base 
#### It will freeze the weights of the layers to prevent them from being updated while trainig

In [ ]:
base_model.trainable = False

#### layers.trainable is kept false because, at the time of batchNormalization,it may update variances and mean. So,this updates may distroy the things that our model has learned

In [ ]:
 base_model.summary()

##### Add a clasification head 

###### Average over the spatial 5x5 spatial locations


#### it converts (32 , 5,5,1280) into 32,1280
#### using GlobalAveragePooling2D
To generate predictions from the block of features, average over the spatial 5x5 spatial locations, using a tf.keras.layers.GlobalAveragePooling2D layer to convert the features to a single 1280-element vector per image.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

### convert these feature into single prediction per image


In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

#### build the model chaining together the data augmentation

In [ ]:
inputs  = tf.keras.Input(shape = (160,160,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x,training = False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs,outputs)

#### Compile the model

In [ ]:
base_learning_rate = 0.001
model.compile(optimizer = tf.keras.optimizers.Adam(lr = base_learning_rate),
             loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
             metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

##### out of 2.5M parameters , only 1.2K are used trainable where other are
frozen

##### Train the model

In [ ]:
initial_epochs = 10
loss0,accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print("Initial Loss: {:.2f}".format(loss0))
print("Initial Accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_dataset,
                   epochs = initial_epochs,
                   validation_data = validation_dataset)

Epoch 1/10
63/63 [==============================] - 49s 727ms/step - loss: 0.3246 - accuracy: 0.8410 - val_loss: 0.1133 - val_accuracy: 0.9579
Epoch 2/10
63/63 [==============================] - 45s 717ms/step - loss: 0.1690 - accuracy: 0.9295 - val_loss: 0.0797 - val_accuracy: 0.9740
Epoch 3/10
63/63 [==============================] - 45s 713ms/step - loss: 0.1480 - accuracy: 0.9365 - val_loss: 0.0692 - val_accuracy: 0.9740
Epoch 4/10
63/63 [==============================] - 45s 720ms/step - loss: 0.1247 - accuracy: 0.9485 - val_loss: 0.0579 - val_accuracy: 0.9752
Epoch 5/10
63/63 [==============================] - 45s 706ms/step - loss: 0.1269 - accuracy: 0.9525 - val_loss: 0.0602 - val_accuracy: 0.9802
Epoch 6/10
63/63 [==============================] - 45s 715ms/step - loss: 0.1268 - accuracy: 0.9440 - val_loss: 0.0534 - val_accuracy: 0.9777
Epoch 7/10
63/63 [==============================] - 45s 716ms/step - loss: 0.1172 - accuracy: 0.9520 - val_loss: 0.0514 - val_accuracy: 0.9790

In [ ]:
## Learning curves


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

#### Fine tuning

#### fine tune is to train the weights on the top of the layers of the pre trained model

### train the weights on the top of pretrained model along with the classifier you added

##### unfreeze the top layer

In [ ]:
base_model.trainable = True

In [ ]:
# let's take a look to see how many layers are in base model
print("Number of layers in base model: ",len(base_model.layers))
## fine tune from this layers onwards
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compile the model


In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits = True),
             optimizer = tf.keras.optimizers.RMSprop(lr = base_learning_rate/10),
             metrics = ['accuracy'])


In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

In [ ]:
### continue the training

In [ ]:
fine_tune_epochs = 10
total_epochs = initial_epochs+fine_tune_epochs
history_fine = model.fit(train_dataset,
                        epochs = total_epochs,
                        initial_epoch = history.epoch[-1],
                        validation_data = validation_dataset)

In [ ]:
acc+=history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']
loss+=history_fine.history['loss']
val_loss+=history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

In [ ]:
### Now test the data

In [ ]:
#Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)

predictions = tf.where(predictions < 0.5, 0, 1)
print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
    
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    plt.title(class_names[predictions[i]])
    plt.axis("off")